In [130]:
# -*- coding: utf-8 -*-

import pdb
import random
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from scipy import stats
from itertools import combinations
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import f_oneway

"""
The function Measure_Patterns has 3 parameters: X_train, y_train, optional
optional will check if the columns selected is categorical (integers and strings) or numerical (float)
if optional is not provided, then the program will assume that the column has integers values, therefore it will be considered categorical
"""



'\nThe function Measure_Patterns has 3 parameters: X_train, y_train, optional\noptional will check if the columns selected is categorical (integers and strings) or numerical (float)\nif optional is not provided, then the program will assume that the column has integers values, therefore it will be considered categorical\n'

In [132]:
# Load dataset 
#data = np.loadtxt("/Users/fabianafazio/Documents/GitHub/BP24/Fabiana/Data/uniform_large_d_1.tex")
#data = np.loadtxt("uniform_small_d_1.tex")
df = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Fabiana/Data/lung_cancer_data.csv")
df.head()

# Creating NumPy array
#array = np.array(data)
# Converting to Pandas DataFrame
#df_table = pd.DataFrame(array)
# Displaying the table
#print(df_table)

,Patient_ID,Age,Gender,Smoking_History,Tumor_Size_mm,Tumor_Location,Stage,Treatment,Survival_Months,Ethnicity,...,Alanine_Aminotransferase_Level,Aspartate_Aminotransferase_Level,Creatinine_Level,LDH_Level,Calcium_Level,Phosphorus_Level,Glucose_Level,Potassium_Level,Sodium_Level,Smoking_Pack_Years
0,Patient0000,68,Male,Current Smoker,81.678677,Lower Lobe,Stage III,Surgery,44,Hispanic,...,27.985571,46.801214,1.245849,239.240255,10.366307,3.547734,113.919243,4.968163,139.822861,17.006956
1,Patient0001,58,Male,Never Smoked,78.448272,Lower Lobe,Stage I,Radiation Therapy,101,Caucasian,...,30.120956,39.711531,1.463231,233.515237,10.081731,2.945020,101.321578,3.896795,135.449361,93.270893
2,Patient0002,44,Male,Former Smoker,67.714305,Lower Lobe,Stage I,Chemotherapy,69,African American,...,5.882418,32.640602,0.630109,169.037460,8.660892,4.637399,78.214177,4.369050,143.377155,70.348376
3,Patient0003,72,Male,Current Smoker,70.806008,Lower Lobe,Stage III,Chemotherapy,95,African American,...,38.908154,44.319393,0.594342,213.967590,8.832669,3.617098,127.895361,4.348474,138.586005,19.828128
4,Patient0004,37,Female,Never Smoked,87.272433,Lower Lobe,Stage IV,Radiation Therapy,105,Asian,...,26.344877,15.746906,1.478239,118.187543,9.247609,4.773255,148.801185,3.671976,141.230724,81.047456


In [134]:
# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:150], df.iloc[:,-1], test_size=0.2, random_state=52)

In [136]:
# Function Measure_Patterns begins here!
def Measure_Patterns(X_train, y_train, optional=None):
    
    # Initialize empty dataframes for numerical and categorical data
    numerical_df = pd.DataFrame()
    categorical_df = pd.DataFrame()
    
    # Check if the data type is provided for columns
    if optional is None:
        print("Optional parameter not provided. Assuming integers values are categorical")
    
        # Splitting X_train into numerical subset 
        print("\nNumerical DataFrame:")
        numerical_df = X_train.select_dtypes(include = ['int', 'int64', 'float', 'float64'])
        print(numerical_df)

        # Splitting X_train into categorical subset 
        print("Categorical DataFrame:")
        categorical_df = X_train.select_dtypes(include=['object', 'category'])
        print(categorical_df)
    
    else:
        # Create empty numerical & categorical data frames
        numerical = []
        numerical_colnames = []
        categorical = []
        categorical_colnames = []
        
        # Check that length of optional = # of columns in X_train
        # Optional is the column type for X_train, so the lengths should be equal
        if len(optional) == len(X_train.columns):
            # For all the values in optional
            for i in range(len(optional)):
                if optional[i] == True:
                    numerical.append(X_train.iloc[:,i])
                    # TO DO: Save SPECIFIC column name in each loop, order matters
                    # numerical_colnames.append()
                else: 
                    categorical.append(X_train.iloc[:,i])
                    # TO DO: Save SPECIFIC column name in each loop, order matters
                    # categorical_colnames.append()
            # Turn transposed arrays into dataframes
            numerical_df = pd.DataFrame(np.transpose(numerical))
            categorical_df = pd.DataFrame(np.transpose(categorical))
            # TO DO: Re-attach the column names to numerical_df & categorical_df 
            
            print("Numerical DF:")
            print(numerical_df)
            print("Categorical Df")
            print(categorical_df)
            
        else:
            print("The length of X_train and optional are different.")

    return numerical_df, categorical_df

# Get numerical and categorical dataframes
#numerical_df, categorical_df = Measure_Patterns(X_train, y_train)
            

In [111]:
##################### Correlation between columns (numerical) Code ############################
# Takes the X_train data to find correlation between NUMERICAL features
def num_corr(X_train_numerical):
    matrix = X_train_numerical.corr(method='pearson')
    print("---------------------------Correlation Matrix------------------------- \n", matrix)
     
    #Calls the function so the matrix prints out    
    num_corr(numerical_df)

In [115]:
####################### Chi-Square (F vs F) Code ################################################
    
print("\n------------------Chi-Squared for Features v. Features-----------------------")
# Finds dependency between all CATEGORICAL features in X_train
def chi_squared_fvf(X_train_categorical):
        
    # Extract variable names
    variable_names = list(X_train_categorical.columns)

    # Initialize matrices to store chi-squared and p-values
    num_variables = len(variable_names)
    chi_squared = np.zeros((num_variables, num_variables))
    p_values = np.zeros((num_variables, num_variables))

    # Compute chi-squared and p-values for each pair of variables
    for i, j in combinations(range(num_variables), 2):
        contingency_table = pd.crosstab(X_train_categorical.iloc[:, i], X_train_categorical.iloc[:, j])
            
        # Compute chi-squared and p-values
        chi2 = stats.chi2_contingency(contingency_table)[0]
        p = stats.chi2_contingency(contingency_table)[1]
            
        # Assign results to chi_squared and p_values matrices
        chi_squared[i, j] = chi2
        chi_squared[j, i] = chi2  # Assign to symmetric position in the matrix
        p_values[i, j] = p
        p_values[j, i] = p  # Assign to symmetric position in the matrix

    # Create a DataFrame with variable names as index and columns
    chi_squared_df = pd.DataFrame(chi_squared, index=variable_names, columns=variable_names)
    p_values_df = pd.DataFrame(p_values, index=variable_names, columns=variable_names)

    # Printing the matrix-like output with variable names
    print("Chi-Squared Values:")
    print(chi_squared_df)
    print("\nP-Values:")
    print(p_values_df)
    
chi_squared_fvf(categorical_df)


------------------Chi-Squared for Features v. Features-----------------------
Chi-Squared Values:
                                          Age      Gender  Smoking_History  \
Age                                  0.000000   49.765405       102.277460   
Gender                              49.765405    0.000000         0.104352   
Smoking_History                    102.277460    0.104352         0.000000   
Tumor_Location                      83.929733    1.606470         2.339686   
Stage                              119.351373    1.752805        15.117773   
Treatment                          117.883856    3.401001         2.837152   
Survival_Months                   5934.278887  111.774555       223.943437   
Ethnicity                          179.875265    4.907301         4.485437   
Insurance_Type                     154.765765    1.599956         9.182261   
Family_History                      59.270783    0.021776         6.433774   
Comorbidity_Diabetes                49.9617

In [ ]:
##################### Chi-Square (F vs label column) Code ####################################
    
    print("\n------------------------Chi-Square (F vs label column)------------------------")
    # Finds dependency between all CATEGORICAL features in X_train & the label in y_train
    def chi_squared_fvl(X_train_categorical, y_train):
        
        # Combining CATEGORICAL X_train and y_train
        df = X_train_categorical
        df['label'] = y_train

        # Number of features, excluding label
        var_count = len(df.columns)-1

        # Creates an empty array to print values in a table
        results = []

        for i in range(0, var_count):

            # Create contigency table of all features v. label
            crosstab = pd.crosstab(df.iloc[:, i], df.iloc[:,-1])
            
            # Compute chi-squared and p-values
            chi2 = stats.chi2_contingency(crosstab)[0]
            p = stats.chi2_contingency(crosstab)[1]
            
            # Append results to the list
            results.append({
                "Feature": df.columns[i],
                "Chi Squared Statistic": chi2,
                "P-Value": p})

        # Create a DataFrame from the results
        results_df = pd.DataFrame(results)

        # Print the DataFrame
        print("Label:", df.columns.values[-1])
        print(results_df.to_string(index=False))
    
    chi_squared_fvl(categorical_df, y_train)

In [138]:
############################# ANOVA (Feature vs Feature) #########################################

print("\n------------------ ANOVA (Feature vs Feature) -----------------------")
# Initialize an empty DataFrame to store ANOVA results
anova_results = pd.DataFrame(index=categorical_df.columns, columns=numerical_df.columns)

# Perform ANOVA for each combination of categorical and numerical columns
for cat_col in categorical_df:
    for num_col in numerical_df:
        groups = []
        for category in X_train[cat_col].unique():
            groups.append(X_train[num_col][X_train[cat_col] == category])
        f_statistic, p_value = f_oneway(*groups)
        anova_results.loc[cat_col, num_col] = f_statistic, p_value

# Create a formatted DataFrame for ANOVA results with labeled interpretation
formatted_results = pd.DataFrame(index=categorical_df.columns, columns=numerical_df.columns)
for num_col in numerical_df:
    for cat_col in categorical_df:
        f_statistic, p_value = anova_results.loc[cat_col, num_col]
        if p_value < 0.05:
            significance = "Significant"
        else:
            significance = "Not Significant"
        formatted_results.loc[cat_col, num_col] = f"F = {f_statistic:.2f}, p = {p_value:.4f} ({significance})"

# Display formatted ANOVA results as a table
print("ANOVA Results:")
print(formatted_results)


------------------ ANOVA (Feature vs Feature) -----------------------
ANOVA Results:
                                                           Tumor_Size_mm  \
Age                               F = 0.79, p = 0.8546 (Not Significant)   
Gender                            F = 0.65, p = 0.4199 (Not Significant)   
Smoking_History                   F = 1.54, p = 0.2134 (Not Significant)   
Tumor_Location                    F = 0.86, p = 0.4218 (Not Significant)   
Stage                             F = 0.75, p = 0.5234 (Not Significant)   
Treatment                         F = 0.60, p = 0.6140 (Not Significant)   
Survival_Months                   F = 0.99, p = 0.5077 (Not Significant)   
Ethnicity                         F = 0.21, p = 0.9345 (Not Significant)   
Insurance_Type                    F = 0.73, p = 0.5367 (Not Significant)   
Family_History                    F = 0.93, p = 0.3348 (Not Significant)   
Comorbidity_Diabetes              F = 0.03, p = 0.8527 (Not Significant)   
Co

# ANOVA FEATURE VS LABEL (PENDING) 

In [144]:
############################# ANOVA (Feature vs Label) #############################
print("\n------------------ ANOVA (Feature vs Label) -----------------------")
# Concatenate X_train and y_train into a single DataFrame for ANOVA comparison
data = pd.concat([X_train, y_train], axis=1)

# Automatically detect all columns as features (numerical and categorical)
all_columns = data.columns

# Identify the last column as the target column
target_column = all_columns[-1]

# Initialize an empty DataFrame to store ANOVA results
anova_results = pd.DataFrame(index=[target_column], columns=all_columns)

# Perform ANOVA for each feature column against the target column
for feature_col in all_columns:
    if feature_col == target_column:
        continue  # Skip the target column itself
    groups = [data[feature_col][data[target_column] == value] for value in data[target_column].unique()]
    f_statistic, p_value = f_oneway(*groups)
    anova_results.loc[target_column, feature_col] = f"F = {f_statistic:.2f}, p = {p_value:.4f}"

# Print ANOVA results for each feature with significance indication
print("ANOVA Results:")
for feature_col in anova_results.columns[:-1]:  # Exclude the last column (target column)
    result = anova_results.loc[target_column, feature_col]
    f_statistic, p_value = result.split(', ')
    f_statistic = float(f_statistic.split(' = ')[1])
    p_value = float(p_value.split(' = ')[1])
    
    if p_value < 0.05:
        significance = "Significant"
    else:
        significance = "Not Significant"
    
    print(f"{feature_col}: {result} ({significance})")



------------------ ANOVA (Feature vs Label) -----------------------


AttributeError: 'DataFrame' object has no attribute 'unique'

In [66]:
########################## Histogram/Graphing ###############################

print("------------------------Histogram/Graphing-----------------------------")


###### These are just here for now so the histogram and bar graph functions work
# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ["float64"])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float64'])
        

# Ensure data is 2D
if numerical_df.ndim == 1:
    numerical_df = numerical_df.reshape(-1, 1)  # Reshape 1D array to 2D array with one column

# Number of features (columns) in the dataset
numerical_num_features = numerical_df.shape[1]


# Loop through each numerical feature
for feature_idx in range(numerical_num_features):
    # Extract the current feature data (column)
    feature_df = numerical_df.iloc[:, feature_idx]

    # Compute histogram with 10 bins
    hist, bin_edges = np.histogram(feature_df, bins=10)

    # Print feature number
    print(f"Feature {feature_idx + 1}:")
    
    # Print bin edges
    print("Bin Edges:", bin_edges)

    # Store bin heights in a list
    bin_heights = []
    bin_heights.extend(hist)
    print("Array with bin heights:", bin_heights)

    # Store bin probabilities in a list and normalize
    bin_probs = []
    bin_probs.extend(hist)
    bin_probs = np.array(bin_probs) / sum(bin_heights)
    print("Array with bin probabilities:", bin_probs)

    # Loop through each bin to print range and probabilities
    for i in range(len(hist)):
        bin_range = f"{bin_edges[i]:.2f} to {bin_edges[i+1]:.2f}"  # Bin range
        bin_probability = hist[i] / sum(hist)  # Bin probability
        print(f"Bin {i + 1} ({bin_range}): Height = {hist[i]}, Probability = {bin_probability:.2f}")

    # Separator between features for clarity
    print("\n" + "="*50 + "\n")

# Calculate and store probabilities for each categorical column
print("Proportions for Label for Categorical Columns:")

for column in categorical_df.columns:
    value_counts = categorical_df[column].value_counts(normalize=True).sort_index()
    # print(f"Probabilities for Categorical Column {column}:")
    print(value_counts)
    print()  # Add an empty line for separation   

------------------------Histogram/Graphing-----------------------------
Feature 1:
Bin Edges: [10.00427851 19.00290603 28.00153355 37.00016107 45.99878859 54.99741611
 63.99604363 72.99467115 81.99329867 90.99192619 99.99055371]
Array with bin heights: [1851, 1881, 1857, 1900, 1938, 1838, 1829, 1900, 1999, 1933]
Array with bin probabilities: [0.09780197 0.09938709 0.09811899 0.100391   0.10239882 0.09711508
 0.09663954 0.100391   0.1056219  0.10213463]
Bin 1 (10.00 to 19.00): Height = 1851, Probability = 0.10
Bin 2 (19.00 to 28.00): Height = 1881, Probability = 0.10
Bin 3 (28.00 to 37.00): Height = 1857, Probability = 0.10
Bin 4 (37.00 to 46.00): Height = 1900, Probability = 0.10
Bin 5 (46.00 to 55.00): Height = 1938, Probability = 0.10
Bin 6 (55.00 to 64.00): Height = 1838, Probability = 0.10
Bin 7 (64.00 to 72.99): Height = 1829, Probability = 0.10
Bin 8 (72.99 to 81.99): Height = 1900, Probability = 0.10
Bin 9 (81.99 to 90.99): Height = 1999, Probability = 0.11
Bin 10 (90.99 to 99.9

In [48]:
############################ KL Divergence ####################################

optional_test = []

for i in range(151):
    optional_test.append(bool(random.getrandbits(1)))

# Call the measure_patterns function
Measure_Patterns(X_train, y_train)

Optional parameter not provided. Assuming integers values are categorical

Numerical DataFrame:
          26        27        28        29        30        31        32   \
159  2.737286  2.564694  2.078000  2.701250  2.172147  2.306469  2.833275   
198  0.449442  0.769937  0.043365  0.899528  0.743107  0.055306  0.189964   
259  2.730227  2.558789  2.801426  2.055368  2.278217  2.426925  2.431457   
301  2.233624  2.514446  2.930156  2.916944  2.435773  2.614566  2.833691   
220  0.687277  0.112971  0.248352  0.483834  0.189635  0.360817  0.808924   
..        ...       ...       ...       ...       ...       ...       ...   
86   2.465128  2.600074  2.676940  2.477864  2.639337  2.902997  2.084804   
151  0.688109  0.625346  0.802530  0.602819  0.343456  0.936555  0.263959   
13   0.839154  0.999741  0.972734  0.602158  0.763564  0.515336  0.740383   
267  0.927585  0.265717  0.361995  0.526540  0.688182  0.129511  0.233144   
156  2.972086  2.940795  2.572536  2.690416  2.993065  2.